## 마지막 퀴즈 정리

In [36]:
import numpy as np 
import pandas as pd
from datetime import date

#원금과 Fixed_rate 설정
National = 10000000
fixedRate = 0.05

# a x b = c 라고 할 때
# payoff = a
Payoff = np.matrix([[102, 0, 0],
                    [2.25,102.25,0], 
                    [2.5,2.5,102.5]])
# price = c
price = np.matrix([100., 100., 100.]).reshape((3,1))

# 무엇을 곱해야 c가 될까? = DF
# b = a^-1 * C
DF=Payoff.I * price

### 1단계 초기값 완성! ###

# 처음 0 기간일 때 DF는 없기 때문에 한칸 삽입
DF = np.concatenate((np.matrix([1]), DF), axis=0)

# date index 넣기
Dates = date(2023,5,10),date(2023,11,10), date(2024,5,10), date(2024,11,10)
df = pd.DataFrame(data = DF , columns=['DF'], index=Dates)

# Days_diff 구하기
df.reset_index(inplace=True)
df.rename(columns={'index':'Date'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df['Date_lag']=df['Date'].shift(1)
df['Days_diff']=(df['Date']-df['Date_lag']).dt.days
df = df[['Date', 'Date_lag','Days_diff', 'DF']]

# ForwardRate를 위한 DF_Lag 구하기 > ForwardRate까지
df['DF_Lag'] = df['DF'].shift(1)
df['ForwardRate'] = (365/df['Days_diff'])*(df['DF_Lag']/df['DF']-1)

### 2단계 Forward Rate까지 완성! ###


# 변동금리 구하기 Cash_flow
CF = [np.nan, np.nan]
for i in range(2,4):
    CF.append(Notional* df.loc[i,'ForwardRate']*df.loc[i,'Days_diff']/365)
df['CF_float']=CF
df['CF_float_PV']=df['CF_float']*df['DF']


# 고정금리 구하기 Cash_flow
CF = [np.nan, np.nan]
for i in range(2,4):
    CF.append(Notional* fixedRate*df.loc[i,'Days_diff']/365)
df['CF_fixed']=CF
df['CF_fixed_PV']=df['CF_fixed']*df['DF']


# 수취 및 지불 구현하기
Receive = df.loc[2:3,'CF_fixed_PV'].sum()
Pay = df.loc[2:3,'CF_float_PV'].sum()
print(Receive-Pay)
df

-4776.708079274707


,Date,Date_lag,Days_diff,DF,DF_Lag,ForwardRate,CF_float,CF_float_PV,CF_fixed,CF_fixed_PV
0,2023-05-10,NaT,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-11-10,2023-05-10,184.0,0.980392,1.000000,0.039674,NaN,NaN,NaN,NaN
2,2024-05-10,2023-11-10,182.0,0.956422,0.980392,0.050263,25062.656642,23970.468383,24931.506849,23845.033877
3,2024-11-10,2024-05-10,184.0,0.928370,0.956422,0.059939,30215.627991,28051.294464,25205.479452,23400.020890


---
## Compounded SOFR
- **source**: 스왑 실무 가이드 by 정창영, pp.67-69.

- 이자계산 기간 동안 고시되는 일별 SOFR 이자를 compounding 하여 연율로 표시
- 초일산입/말일불산입 원칙 적용
- Compounding 기본식
$$ \left[ \prod_{i=1}^{d_0} \left( 1+ \frac{r_i \times n_i}{\text{Day Count Basis}} \right) -1\right] \times \frac{\text{Day Count Basis}}{d}$$
    - $r_i$: underlying benchmark level at day $i$
    - $n_i$: SOFR 이자율이 고시되지 않는 날을 고려하여 해당 날 수($n_i$) 만큼 $r_i$를 적용

**Example**: 2022년 10월 8일부터 10월 16일까지

<img src="https://raw.github.com/zara2k/lecture/master/Swap_Fig07_SOFR_example_calendar.png" width=500>

- 8일~10일: 이자계산 초일이 영업일이 아니므로 전 영업일 SOFR 금리를 적용
$$1+ \frac{0.11 \% \times 3}{360}$$
- 11일 (영업일)
$$1+ \frac{0.12 \% }{360}$$
- 12일 (영업일)
$$1+ \frac{0.13 \% }{360}$$
- 13일 (영업일)
$$1+ \frac{0.14 \% }{360}$$
- 14일~16일: 이자계산 종료일인 16일은 초일산입/말일불산입에 따라 제외
$$1+ \frac{0.15 \% \times 2}{360}$$

In [1]:
import numpy as np
r = np.array([0.11, 0.12, 0.13, 0.14, 0.15])
n = np.array([   3,    1,    1,    1,    2])

d = r.shape[0]


CompoundedSOFR = 1
for i in range(d):
    CompoundedSOFR*=(1+ r[i]/100*n[i]/360)
    
CompoundedSOFR -= 1
CompoundedSOFR = CompoundedSOFR*360/sum(n)
print(f"{CompoundedSOFR*100:.5}%")

5
0.1275%


---
- 용어 
    1. Trade Date : FRA 계약일
    1. Contract Amount : 예금/차입하려는 금액, 원금
    1. Fixing Date : 변동금리 결정되는 날짜, Settlement Date의 2영업일 전
    1. Fixing Rate : 결정된 변동금리 
    1. Settlement Date : FRA 계약이 시작되는 날짜(또는 명목상의 예금/대출이 시작되는 날짜), 현금결제 금액을 정산하는 날짜
    1. Maturity Date : 명목상 예금/대출의 만기일 
    1. Contract Rate : FRA 계약금리 
    1. Contract Period : FRA 계약기간,  Settlement Date 와 Maturity Date  사이의 기간 

<img src="https://raw.github.com/zara2k/lecture/master/FRA_Fig02_2by8.png" width=500>

---
**Example) FRA Libor**
- 증권회사 C는 1x4 USD FRA의 가격이 2.3%-2.4%일 때 고객에게 FRA 매도
- 계약상세
    - Notional : USD 10m
    - Trade Date : 2023.3.28.
    - Fixing Date : 2023.4.28.
    - Settlement Date : 2023.4.30.
    - Maturity Date : 2023.7.30.
- 만약 4월 28일 fixing rate이 2.5%라면 C의 결제금액은?
    - C는 market maker이고 FRA 매도 거래를 하였으므로 Contract Rate은 2.4%
    - 3m Libor가 2.5%이므로 2.5%지급, 2.4% 수취. 즉 0.1% 고객에게 지급 
    - Maturity Date(2023.7.30) 결제금액은 USD 10m x 0.001 x 91/360 = USD 2,527.78
    - Settlement Date에 결제하는것이 관행이므로 2.5%로 91일 동안 할인한  USD 2,511.90을 Settlement Date(2023.4.30.)에 지급 

In [1]:
import datetime
datetime.date(2023,7,30)-datetime.date(2023,4,30)

datetime.timedelta(days=91)

In [4]:
10000000 * 0.001 * 91/360 

2527.777777777778

In [5]:
10000000 * 0.001 * 91/360 /(1+0.025*91/360)

2511.9039403767856

---
**Pricing of FRN**

- FRN이 발행되는 시점에서 미래에 지급될 이자를 모르기때문에 가격계산이 어려울 듯이 보이나 실상은 그렇지 않음.
- 쿠폰이자율이 시장이자율에 따라 변동하기 때문에 가격은 항상 par에 가깝게 됨.
- 6개월마다 6m CD 지급하는 2년만기 FRN

<img src="https://raw.github.com/zara2k/lecture/master/FRN_Fig01_pricing.png" width=500>

- 만기 6개월 전인 1.5 시점에서의 ex-coupon FRN의 가치
$$\frac{100 \left(1+i_{1.5} \times \frac{days}{365}\right)}{1+i_{1.5} \times \frac{days}{365}}=100$$
- 만기 1년 전인 1.0 시점에서의 ex-coupon FRN의 가치
$$\frac{100 \left(1+i_{1.0} \times \frac{days}{365}\right)}{1+i_{1.0} \times \frac{days}{365}}=100$$
- 이렇게 계속 하면 reset date에서의 FRN의 가치는 항상 par value, 즉, 100이 됨
- reset dates $t_a$, $t_b$ 의 중간시점 t에서의 FRN 가치는, (아래에서 $i_t$는 $t$시점의 LIBOR 금리, num는 $t$시점과 $t_b$ 시점 사이의 날짜 수
$$\frac{100 \left(1+i_{a} \times \frac{days}{365}\right)}{1+i_{t} \times \frac{num}{365}}=100$$

---

- **FX forward contracts** : Agreements to buy and sell foreign currencies in the future at prices, $F_{0,t}$, agreed upon today (at time $0$)
- **Covered Interest Parity (CIP)** : Under no-arbitrage condition, the fair forward FX rate is (with European quote)
\begin{eqnarray}
F_{0,t} &=& S_0 \left( \frac{1+r_{KRW} \times t}{1+r_{USD} \times t} \right)\\
&=& 1000 \left( \frac{1+0.071 \times 1}{1+0.02 \times 1} \right) = 1,050 (KRW/USD)
\end{eqnarray}

<img src="https://raw.github.com/zara2k/lecture/master/FxSwap_Fig02_CIP.bmp" width=500>

---
**Example**
- 미국계 A은행이 한국 B은행과 현물환율 1,200원, 1년 선도환율 1217.96원에 1백만달러의 현물환매도/선물환매입(sell & buy)    
- 최초 거래시점에서 A은행은 B은행에 1백만달러를 지급하는 대가로 12억원(1200원x 1 백만달러)을 수취
- 이후 만기시점에서 A은행은 B은행으로부터 1백만달러를 돌려 받고 원화 12.1796억원(1217.96원 x 1백만달러)을 상환함으로써 거래가 종결
- 이 경우 A은행은 단기적으로 부족한 원화유동성을, B은행은 외화유동성을 각각 확보

- Sell & Buy FX Swap

.|.
---|---
현물환율($S_0$) | KRW/USD 1,200.0
USD 1년 이자율 | 3.55%, A/360
KRW 1년 이자율 | 5.15%, A/365F
1년 선도환율($F_{0,1}$)| KRW/USD 1,217.96(이론가격=시장가격)

In [7]:
1200 * (1+0.0515 * 365/365)/(1+0.0355 * 365/360)

1217.961832112238

---

- Position 관련 용어
    - Buyer = fixed rate payer = long a swap = shorting the bond
    - Seller = floating rate payer = shorting a swap = long a bond

---

In [36]:
import numpy as np 
import pandas as pd
from datetime import date

#원금과 Fixed_rate 설정
National = 10000000
fixedRate = 0.05

# a x b = c 라고 할 때
# payoff = a
Payoff = np.matrix([[102, 0, 0],
                    [2.25,102.25,0], 
                    [2.5,2.5,102.5]])
# price = c
price = np.matrix([100., 100., 100.]).reshape((3,1))

# 무엇을 곱해야 c가 될까? = DF
# b = a^-1 * C
DF=Payoff.I * price

### 1단계 초기값 완성! ###

# 처음 0 기간일 때 DF는 없기 때문에 한칸 삽입
DF = np.concatenate((np.matrix([1]), DF), axis=0)

# date index 넣기
Dates = date(2023,5,10),date(2023,11,10), date(2024,5,10), date(2024,11,10)
df = pd.DataFrame(data = DF , columns=['DF'], index=Dates)

# Days_diff 구하기
df.reset_index(inplace=True)
df.rename(columns={'index':'Date'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df['Date_lag']=df['Date'].shift(1)
df['Days_diff']=(df['Date']-df['Date_lag']).dt.days
df = df[['Date', 'Date_lag','Days_diff', 'DF']]

# ForwardRate를 위한 DF_Lag 구하기 > ForwardRate까지
df['DF_Lag'] = df['DF'].shift(1)
df['ForwardRate'] = (365/df['Days_diff'])*(df['DF_Lag']/df['DF']-1)

### 2단계 Forward Rate까지 완성! ###


# 변동금리 구하기 Cash_flow
CF = [np.nan, np.nan]
for i in range(2,4):
    CF.append(Notional* df.loc[i,'ForwardRate']*df.loc[i,'Days_diff']/365)
df['CF_float']=CF
df['CF_float_PV']=df['CF_float']*df['DF']


# 고정금리 구하기 Cash_flow
CF = [np.nan, np.nan]
for i in range(2,4):
    CF.append(Notional* fixedRate*df.loc[i,'Days_diff']/365)
df['CF_fixed']=CF
df['CF_fixed_PV']=df['CF_fixed']*df['DF']


# 수취 및 지불 구현하기
Receive = df.loc[2:3,'CF_fixed_PV'].sum()
Pay = df.loc[2:3,'CF_float_PV'].sum()
print(Receive-Pay)
df

-4776.708079274707


,Date,Date_lag,Days_diff,DF,DF_Lag,ForwardRate,CF_float,CF_float_PV,CF_fixed,CF_fixed_PV
0,2023-05-10,NaT,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-11-10,2023-05-10,184.0,0.980392,1.000000,0.039674,NaN,NaN,NaN,NaN
2,2024-05-10,2023-11-10,182.0,0.956422,0.980392,0.050263,25062.656642,23970.468383,24931.506849,23845.033877
3,2024-11-10,2024-05-10,184.0,0.928370,0.956422,0.059939,30215.627991,28051.294464,25205.479452,23400.020890
